In [80]:
import pandas as pd
from datetime import datetime, timedelta

pd.set_option('display.max_rows', 500)

# Adds 8 hours to timestamp (for GMT-8 tz) to make GMT time. Replace with your offset from GMT.
TZ_ADJ = 8-3

data_path = './data.csv'

In [75]:
def rsi(df, periods=14):
    """
    Returns a pd.Series with the relative strength index.
    From https://www.roelpeters.be/many-ways-to-calculate-the-rsi-in-python-pandas/
    """
    close_delta = df['close'].diff()

    # Make two series: one for lower closes and one for higher closes
    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)
    
    # Use exponential moving average
    ma_up = up.ewm(com=periods-1, adjust=True, min_periods=periods).mean()
    ma_down = down.ewm(com=periods-1, adjust=True, min_periods=periods).mean()
        
    rsi = ma_up / ma_down
    rsi = 100 - (100/(1 + rsi))
    return rsi

In [76]:
data = pd.read_csv(data_path)

# adjusting from GMT-8 local tz
data['time'] = data['time'].apply(lambda x: datetime.fromtimestamp(x) + timedelta(seconds=60*60*TZ_ADJ))

data['rsi'] = rsi(data)

In [59]:
# Find where RSI goes below 30 or above 70, and then retracts on the next bar. 
# Using the close of that bar as the entry, what is the MAE and MFE of the next 4 bars? 


In [77]:
def calculate_indicators(df,lower=30,upper=70,periods_shift=4):

    df["rsi_t+1"]=df["rsi"].shift(-1)
    condition1=(df["rsi"]<lower) & (df["rsi_t+1"]>lower)
    condition2=(df["rsi"]>upper) & (df["rsi_t+1"]<upper)

    enter_indexes=df[(condition1) | (condition2)].index

    for i in range(1,periods_shift+1):
        df["low_"+str(i)]=df["low"].shift(-i)
        df["high_"+str(i)]=df["high"].shift(-i)
        
    low_cols= [col for col in df if col.startswith('low_')]
    high_cols= [col for col in df if col.startswith('high_')]
    
    df["lowest_" + str(periods_shift)]=df[low_cols].min(axis=1)
    df["highest_"+str(periods_shift)]=df[high_cols].max(axis=1)
    
    df["MAE"]=((df["close"]-df["lowest_"+str(periods_shift)])/data["close"])*100
    df["MFE"]=((df["close"]-df["highest_"+str(periods_shift)])/data["close"])*100

    df=df[df.index.isin(enter_indexes)][["time","open","high","low","close","rsi","MAE","MFE"]]

    return df

In [78]:
final_data=calculate_indicators(data)

In [81]:
final_data

,time,open,high,low,close,rsi,MAE,MFE
14,2020-07-21 20:00:00,1.27627,1.27681,1.27513,1.27581,75.885488,0.340960,-0.033704
96,2020-07-27 06:00:00,1.28423,1.28459,1.28359,1.28379,72.452000,0.243030,-0.033495
103,2020-07-27 13:00:00,1.28707,1.28771,1.28559,1.28726,75.448797,0.243929,-0.233830
106,2020-07-27 16:00:00,1.28728,1.29027,1.28627,1.28973,74.163082,0.228730,-0.028688
133,2020-07-28 19:00:00,1.29478,1.29526,1.29395,1.29487,70.734816,0.193070,-0.013129
149,2020-07-29 11:00:00,1.29664,1.29757,1.29611,1.29683,72.403177,0.079424,-0.205116
153,2020-07-29 15:00:00,1.29792,1.29949,1.29677,1.29854,75.369921,0.233339,-0.063148
178,2020-07-30 16:00:00,1.30444,1.30725,1.30311,1.30440,71.774708,0.180159,-0.356486
201,2020-07-31 15:00:00,1.31533,1.31623,1.31243,1.31611,78.015080,0.612411,-0.069143
248,2020-08-04 14:00:00,1.30242,1.30277,1.29814,1.29857,27.980812,0.024642,-0.649176
